Тестовый файл для удобного запуска и отладки составных частей бота 

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
from tinkoff.invest import Client

load_dotenv()

TOKEN = os.getenv("INVEST_TOKEN")

TICKER = 'RNFT'

In [3]:
from trader_bot import ScalpingBot

In [4]:
bot = ScalpingBot(TOKEN, TICKER)

2024-03-16 00:46:33,130 - INFO - INIT 
     figi - BBG00F9XX7H4 (RNFT)
     candles_count - 4
     min profit - 5 steps * 0.1 = 0.5
     stop_loss_percent - 1.0 %
     commission - 0.05 %
     no_operation_timeout_seconds - 300 sec
     sleep_trading - 300 sec
     sleep_no_trade - 300 sec



In [5]:
# bot.db_add_deal(1, 10, 1, 9)

In [6]:
from tinkoff.invest import CandleInterval
from datetime import datetime, timezone

# CandleInterval.CANDLE_INTERVAL_1_MIN
# CandleInterval.CANDLE_INTERVAL_5_MIN

tmz = 3

year = 2024
month = 3
day = 15

date_from = datetime(year, month, day, 10 - 1 - tmz, 60 - 5)
date_to   = datetime(year, month, day, 19 - tmz, 0)

with Client(TOKEN) as client:
    candles_1_min = client.market_data.get_candles(
        figi=bot.client.figi,
        from_=date_from,
        to=date_to,
        interval=CandleInterval.CANDLE_INTERVAL_1_MIN
    )
    
    candles_5_min = client.market_data.get_candles(
        figi=bot.client.figi,
        from_=date_from,
        to=date_to,
        interval=CandleInterval.CANDLE_INTERVAL_5_MIN
    )

In [7]:
from tinkoff.invest import Quotation


def f2q(price) -> Quotation:
    return Quotation(units=int(price), nano=int((round(price - int(price), 1)) * 1e9))

def q2f(quotation: Quotation, digits=1):
    return round(quotation.units + quotation.nano * 1e-9, digits)

In [8]:
candles_1_min_dict = {(candle.time.hour + tmz, candle.time.minute): candle for candle in candles_1_min.candles}
candles_5_min_dict = {(candle.time.hour + tmz, candle.time.minute): candle for candle in candles_5_min.candles}

In [9]:
t5 = candles_5_min_dict[(10, 0)]
t5

HistoricCandle(open=Quotation(units=173, nano=200000000), high=Quotation(units=173, nano=600000000), low=Quotation(units=172, nano=0), close=Quotation(units=172, nano=900000000), volume=97993, time=datetime.datetime(2024, 3, 15, 7, 0, tzinfo=datetime.timezone.utc), is_complete=True)

In [10]:
t5.open

Quotation(units=173, nano=200000000)

In [11]:
print(f"o{q2f(t5.open)} c{q2f(t5.close)} h{q2f(t5.high)} l{q2f(t5.low)} v{t5.volume}")

o173.2 c172.9 h173.6 l172.0 v97993


In [12]:
t1 = candles_1_min_dict[(10, 0)]
print(f"o{q2f(t1.open)} c{q2f(t1.close)} h{q2f(t1.high)} l{q2f(t1.low)} v{t1.volume}")

o173.2 c173.4 h173.6 l173.2 v6425


In [13]:
from tinkoff.invest import HistoricCandle
from datetime import datetime, timedelta

def get_n_minutes(h, m, n):
    specified_time = datetime.now().replace(hour=h, minute=m, second=0, microsecond=0)
    prev_minutes = []
    
    # Вычитаем минуты и добавляем результат в список
    for i in range(n):
        previous_time = specified_time + timedelta(minutes=i)
        prev_minutes.append((previous_time.hour, previous_time.minute))
        
    return prev_minutes

# Пример использования
hour, minute, n = 10, 30, 5
t5 = candles_5_min_dict[(hour, minute)]

def get_candle(hour, minute, n = 5):

    previous_minutes = get_n_minutes(hour, minute, n)
    
    open = None
    high = 0
    low = 1000000000
    close = 0
    volume = 0

    for time_pair in previous_minutes:
        t1 = candles_1_min_dict[time_pair]
        if open is None:
            open = t1.open
        high = max(high, q2f(t1.high))
        low  = min(low,  q2f(t1.low))
        close = t1.close
        volume += t1.volume
        
    return HistoricCandle(
        high=f2q(high),
        low=f2q(low),
        open=open,
        close=close,
        volume=volume,
        time=datetime(year, month, day, hour, minute)
    )

def print_candle(candle:HistoricCandle):
    print(f"o{q2f(candle.open)} c{q2f(candle.close)} h{q2f(candle.high)} l{q2f(candle.low)} v{candle.volume}")
    
t5   = candles_5_min_dict[(hour, minute)]     
t5_c = get_candle(hour, minute, n)

print_candle(t5)
print_candle(t5_c)


o174.3 c174.4 h174.5 l173.7 v52270
o174.3 c174.4 h174.5 l173.7 v52270
